In [165]:
import pandas as pd
import numpy as np
import os
import pickle
import glob
import tqdm
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [57]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [173]:
Dataset_=os.listdir("C:/Users/PARIKALP/Downloads/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/")
Dataset="C:/Users/PARIKALP/Downloads/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/"
Dataset_train=os.listdir("C:/Users/PARIKALP/Downloads/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train/")
Dataset_test=os.listdir("C:/Users/PARIKALP/Downloads/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/")
Dataset_train,Dataset_test

(['NORMAL', 'PNEUMONIA'], ['NORMAL', 'PNEUMONIA'])

In [174]:
train_norm=os.listdir("C:/Users/PARIKALP/Downloads/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train/NORMAl")
train_covid=os.listdir("C:/Users/PARIKALP/Downloads/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train/PNEUMONIA")

In [175]:
test_norm=os.listdir("C:/Users/PARIKALP/Downloads/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/NORMAl")
test_covid=os.listdir("C:/Users/PARIKALP/Downloads/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA")

In [176]:
def dataframe(filenames,classes):
    return pd.DataFrame({"Filename":filenames,"Class":[classes]*len(filenames)})

train_norm_df=dataframe(train_norm,"Normal")
train_covid_df=dataframe(train_covid,"Covid")

test_norm_df=dataframe(test_norm,"Normal")
test_covid_df=dataframe(test_norm,"Covid")

train_df=pd.concat([train_norm_df,train_covid_df],axis=0)
test_df=pd.concat([test_norm_df,test_covid_df],axis=0)

In [177]:
train_df.head()

,Filename,Class
0,IM-0001-0001.jpeg,Normal
1,IM-0003-0001.jpeg,Normal
2,IM-0005-0001.jpeg,Normal
3,IM-0006-0001.jpeg,Normal
4,IM-0007-0001.jpeg,Normal


In [178]:
train_covid_df.head()

,Filename,Class
0,01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg,Covid
1,1-s2.0-S0140673620303706-fx1_lrg.jpg,Covid
2,1-s2.0-S1684118220300608-main.pdf-001.jpg,Covid
3,1-s2.0-S1684118220300608-main.pdf-002.jpg,Covid
4,1-s2.0-S1684118220300682-main.pdf-002-a1.png,Covid


In [179]:
def pathmaker(df_name,df,empty_list):
    for i in df_name.values:
        if i[1]=="Normal":
            empty_list.append(str(Dataset+df+"/NORMAL/"+i[0]))
        else:
            empty_list.append(str(Dataset+df+'/PNEUMONIA/'+i[0]))

train_path=[]
test_path=[]

pathmaker(train_df,"train",train_path)
pathmaker(test_df,"test",test_path)

train_df["Path"]=train_path
test_df["Path"]=test_path

#Shuffling / Re-arranging rows
train_df = train_df.sample(frac=1).reset_index(drop=True).iloc[:, 1:]
test_df = test_df.sample(frac=1).reset_index(drop=True).iloc[:, 1:]

train_df.head()

,Class,Path
0,Covid,C:/Users/PARIKALP/Downloads/covid19-xray-datas...
1,Covid,C:/Users/PARIKALP/Downloads/covid19-xray-datas...
2,Normal,C:/Users/PARIKALP/Downloads/covid19-xray-datas...
3,Covid,C:/Users/PARIKALP/Downloads/covid19-xray-datas...
4,Normal,C:/Users/PARIKALP/Downloads/covid19-xray-datas...


In [180]:
train_df["Path"]

0      C:/Users/PARIKALP/Downloads/covid19-xray-datas...
1      C:/Users/PARIKALP/Downloads/covid19-xray-datas...
2      C:/Users/PARIKALP/Downloads/covid19-xray-datas...
3      C:/Users/PARIKALP/Downloads/covid19-xray-datas...
4      C:/Users/PARIKALP/Downloads/covid19-xray-datas...
                             ...                        
143    C:/Users/PARIKALP/Downloads/covid19-xray-datas...
144    C:/Users/PARIKALP/Downloads/covid19-xray-datas...
145    C:/Users/PARIKALP/Downloads/covid19-xray-datas...
146    C:/Users/PARIKALP/Downloads/covid19-xray-datas...
147    C:/Users/PARIKALP/Downloads/covid19-xray-datas...
Name: Path, Length: 148, dtype: object

In [109]:
# def img_to_mtx(data,empty_array):
#     for i in data:
#         temp_img=image.load_img(i,target_size=(150,150))
#         tem_img=img_to_array(temp_img)
#         empty_array.append(tem_img)

# train_imgs=[]
# test_imgs=[]

# #Applying Image to Matrix
# im=img_to_mtx(train_df['Path'], train_imgs)
# # img_to_mtx(test_df['Path'], test_imgs)



In [110]:
train_imgs

[array([[[1., 1., 1.],
         [3., 3., 3.],
         [3., 3., 3.],
         ...,
         [9., 9., 9.],
         [6., 6., 6.],
         [3., 3., 3.]],
 
        [[2., 2., 2.],
         [2., 2., 2.],
         [3., 3., 3.],
         ...,
         [7., 7., 7.],
         [4., 4., 4.],
         [5., 5., 5.]],
 
        [[2., 2., 2.],
         [3., 3., 3.],
         [4., 4., 4.],
         ...,
         [7., 7., 7.],
         [4., 4., 4.],
         [3., 3., 3.]],
 
        ...,
 
        [[0., 0., 0.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [5., 5., 5.],
         [5., 5., 5.],
         [7., 7., 7.]],
 
        [[0., 0., 0.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [3., 3., 3.],
         [4., 4., 4.],
         [5., 5., 5.]],
 
        [[0., 0., 0.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [5., 5., 5.],
         [7., 7., 7.],
         [4., 4., 4.]]], dtype=float32), array([[[12., 12., 12.],
         [11., 

In [111]:
def img_to_mtx(data):
    
    train_imgs=[]
    for i in data:
        img=cv2.imread(i)
        img=cv2.resize(img,(70,70))
        arr=np.array(img)
        train_imgs.append(arr)
        
img_to_mtx(train_df["Path"])

In [112]:
train_imgs

[array([[[1., 1., 1.],
         [3., 3., 3.],
         [3., 3., 3.],
         ...,
         [9., 9., 9.],
         [6., 6., 6.],
         [3., 3., 3.]],
 
        [[2., 2., 2.],
         [2., 2., 2.],
         [3., 3., 3.],
         ...,
         [7., 7., 7.],
         [4., 4., 4.],
         [5., 5., 5.]],
 
        [[2., 2., 2.],
         [3., 3., 3.],
         [4., 4., 4.],
         ...,
         [7., 7., 7.],
         [4., 4., 4.],
         [3., 3., 3.]],
 
        ...,
 
        [[0., 0., 0.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [5., 5., 5.],
         [5., 5., 5.],
         [7., 7., 7.]],
 
        [[0., 0., 0.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [3., 3., 3.],
         [4., 4., 4.],
         [5., 5., 5.]],
 
        [[0., 0., 0.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [5., 5., 5.],
         [7., 7., 7.],
         [4., 4., 4.]]], dtype=float32), array([[[12., 12., 12.],
         [11., 

In [113]:
def img_to_mtx(data):
    test_imgs=[]
    for i in data:
        img=cv2.imread(i)
        img=cv2.resize(img,(70,70))
        arr=np.array(img)
        test_imgs.append(arr)
#         test_imgs.append(img)
        
img_to_mtx(test_df["Path"])

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [181]:
data=np.concatenate([test_norm,test_covid])

In [182]:
data

array(['NORMAL2-IM-0035-0001.jpeg', 'NORMAL2-IM-0052-0001.jpeg',
       'NORMAL2-IM-0058-0001.jpeg', 'NORMAL2-IM-0059-0001.jpeg',
       'NORMAL2-IM-0072-0001.jpeg', 'NORMAL2-IM-0073-0001.jpeg',
       'NORMAL2-IM-0092-0001.jpeg', 'NORMAL2-IM-0105-0001.jpeg',
       'NORMAL2-IM-0110-0001.jpeg', 'NORMAL2-IM-0111-0001.jpeg',
       'NORMAL2-IM-0112-0001.jpeg', 'NORMAL2-IM-0117-0001.jpeg',
       'NORMAL2-IM-0120-0001.jpeg', 'NORMAL2-IM-0123-0001.jpeg',
       'NORMAL2-IM-0130-0001.jpeg', 'NORMAL2-IM-0131-0001.jpeg',
       'NORMAL2-IM-0132-0001.jpeg', 'NORMAL2-IM-0139-0001.jpeg',
       'NORMAL2-IM-0145-0001.jpeg', 'NORMAL2-IM-0171-0001.jpeg',
       'ryct.2020200034.fig2.jpeg', 'ryct.2020200034.fig5-day0.jpeg',
       'ryct.2020200034.fig5-day4.jpeg', 'ryct.2020200034.fig5-day7.jpeg',
       'SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0.jpeg',
       'SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg',
       'SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10.jpeg',
       'SARS-10.1148r

In [183]:
target_X=np.full(len(test_norm),1)
target_Y=np.full(len(test_covid),0)

In [184]:
target_Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [185]:
target_data=np.concatenate([target_X,target_Y])

In [186]:
target_data

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [187]:
X=[]
for i in test_norm:
    img=cv2.imread("C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/NORMAL/"+i)
    img=cv2.resize(img,(32,32))
    (b,g,r)=cv2.split(img)
    img=cv2.merge([r,g,b])
    X.append(img)

In [188]:
for i in test_covid:
    img=cv2.imread("C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/"+i)
    img=cv2.resize(img,(32,32))
    (b,g,r)=cv2.split(img)
    img=cv2.merge([r,g,b])
    X.append(img)

In [189]:
len(X)

40

In [190]:
len(target_data)

40

In [253]:
X=torch.tensor(X).view(-1,32,32)

c:\users\parikalp\appdata\local\programs\python\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [254]:
X=X/X.max()

In [255]:
X

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.1725, 0.3804, 0.3804],
         [0.3804, 0.2039, 0.2039,  ..., 0.5569, 0.5569, 0.3451],
         [0.3451, 0.3451, 0.1137,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.4275, 0.4275, 0.6078,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.5373, 0.5137, 0.5137],
         [0.5137, 0.4902, 0.4902,  ..., 0.4431, 0.4431, 0.6314]],

        [[0.6314, 0.6314, 0.5765,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.4157, 0.3804, 0.3804],
         [0.3804, 0.3451, 0.3451,  ..., 0.4118, 0.4118, 0.4118],
         ...,
         [0.4078, 0.3294, 0.3294,  ..., 0.6980, 0.6980, 0.6314],
         [0.6314, 0.6314, 0.3686,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.3059, 0.3412, 0.3412]],

        [[0.3412, 0.4863, 0.4863,  ..., 0.7098, 0.7098, 0.6824],
         [0.6824, 0.6824, 0.3412,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.3333, 0.3373, 0.

In [256]:
val_PCL=0.9
val_size=int(len(X)*val_PCL)
val_size

108

In [257]:
Y=target_data

In [258]:
train_x=X[:-val_size]
train_y=Y[:-val_size]
test_x=X[-val_size:]
test_y=Y[-val_size:]

In [259]:
train_x.shape,train_y.shape,test_x.shape,test_y.shape

(torch.Size([12, 32, 32]), (0,), torch.Size([108, 32, 32]), (40,))

In [260]:
len(train_x),len(train_y),len(test_x),len(test_y)

(12, 0, 108, 40)

In [261]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn_1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu_1=nn.ReLU()
        self.max_1=nn.MaxPool2d(kernel_size=2)
        
        self.cnn_2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu_2=nn.ReLU()
        self.max_2=nn.MaxPool2d(kernel_size=2)
        
        self.fc1=nn.Linear(32*8*8,2)
        self.relu_3=nn.ReLU()
        
        
        
    def forward(self,out):
        out=self.cnn_1(out)
        out=self.relu_1(out)
        out=self.max_1(out)
        
        out=self.cnn_2(out)
        out=self.relu_2(out)
        out=self.max_2(out)
        out=self.relu_3(out)
        
        out=out.view(out.size(0),-1)
        out=self.fc1(out)
        return out
    

model=CNN()

In [262]:
model

CNN(
  (cnn_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu_1): ReLU()
  (max_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu_2): ReLU()
  (max_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=2, bias=True)
  (relu_3): ReLU()
)

In [263]:
criterion=nn.MSELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [264]:
ATCH_SIZE = 100
EPOCHS = 100
BATCH_SIZE=100
for epoch in range(EPOCHS):
    for i in tqdm.tqdm(range(0, len(train_x), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_x[i:i+BATCH_SIZE].view(-1, 1, 32,32)
        batch_y = train_y[i:i+BATCH_SIZE]

        model.zero_grad()

        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print(f"Epoch: {epoch}. Loss: {loss.data}")

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'requires_grad'

# Pytorch

In [314]:
IMG_PATH="C:/Users/PARIKALP/AppData/Local/Programs/Python/Scripts/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/"    
class Making():
    def __init__(self):
        self.PATH=IMG_PATH
        self.NO=IMG_PATH+"NORMAL/"
        self.YES=IMG_PATH+"PNEUMONIA/"
        self.LABEL={self.NO:0,self.YES:1}
        self.IMG_SIZE=(70,70)
        self.Training=[]
    
    def make_training_data(self):
        for label in self.LABEL:
            for img in tqdm.tqdm(os.listdir(label)):
                path=os.path.join(label,img)
                img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
                img=cv2.resize(img,(self.IMG_SIZE))
                self.Training.append([np.array(img),np.eye(2)[self.LABEL[label]]])
        
        np.random.shuffle(self.Training)
        np.save("training_data_brain_tumor.npy",self.Training)
        
M=Making()
M.make_training_data()
training_data2=np.load("training_data_brain_tumor.npy",allow_pickle=True)
print(len(training_data2))

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 12.02it/s]


40


In [315]:
x=torch.Tensor([i[0] for i in training_data2]).view(-1,70,70)
X=x/255.0
y=torch.Tensor([i[1] for i in training_data2])

In [316]:
X.shape,y.shape

(torch.Size([40, 70, 70]), torch.Size([40, 2]))

In [317]:
VAL_PCT=0.8
val_size=int(len(X)*VAL_PCT)
val_size

32

In [318]:
train_X=X[:-val_size]
train_Y=y[:-val_size]
test_X=X[-val_size:]
test_Y=y[-val_size:]

In [319]:
len(train_X),len(train_Y),len(test_X),len(test_Y)

(8, 8, 32, 32)

In [320]:
train_X.shape,train_Y.shape,test_X.shape,test_Y.shape

(torch.Size([8, 70, 70]),
 torch.Size([8, 2]),
 torch.Size([32, 70, 70]),
 torch.Size([32, 2]))

In [339]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1=nn.ReLU()
        self.max1=nn.MaxPool2d(kernel_size=2)
            
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu2=nn.ReLU()
        self.max2=nn.MaxPool2d(kernel_size=2)
        
        self.fc1=nn.Linear(32*17*17,1)
        self.relu3=nn.ReLU()
        
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.relu1(out)
        out=self.max1(out)
        
        out=self.cnn2(out)
        out=self.relu2(out)
        out=self.max2(out)
        out=self.relu3(out)
        
        out=out.view(out.size(0),-1)
        out=self.fc1(out)
        return out
        
        
        
model=CNN()
model

CNN(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (max1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (max2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9248, out_features=1, bias=True)
  (relu3): ReLU()
)

In [340]:
criterion=nn.MSELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [341]:
ATCH_SIZE = 100
EPOCHS = 100
BATCH_SIZE=100
for epoch in range(EPOCHS):
    for i in tqdm.tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 70, 70)
        batch_y = train_Y[i:i+BATCH_SIZE]

        model.zero_grad()

        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print(f"Epoch: {epoch}. Loss: {loss.data}")

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Epoch: 0. Loss: 0.4014042019844055
Epoch: 1. Loss: 0.8227556943893433


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Epoch: 2. Loss: 0.6578316688537598
Epoch: 3. Loss: 0.5054287910461426
Epoch: 4. Loss: 0.4758690893650055


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

...

Epoch: 93. Loss: 0.2503235638141632
Epoch: 94. Loss: 0.2503178119659424


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Epoch: 95. Loss: 0.2503122389316559
Epoch: 96. Loss: 0.25030672550201416
Epoch: 97. Loss: 0.250301331281662


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.09it/s]

Epoch: 98. Loss: 0.2502959668636322
Epoch: 99. Loss: 0.2502906620502472


In [389]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1=nn.ReLU()
        self.max1=nn.MaxPool2d(kernel_size=2)
            
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu2=nn.ReLU()
        self.max2=nn.MaxPool2d(kernel_size=2)
        
        self.fc1=nn.Linear(32*17*17,2)
        self.relu3=nn.ReLU()
        
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.relu1(out)
        out=self.max1(out)
        
        out=self.cnn2(out)
        out=self.relu2(out)
        out=self.max2(out)
        out=self.relu3(out)
        
        out=out.view(out.size(0),-1)
        out=self.fc1(out)
        return out
        
        
        
model=CNN()
model

CNN(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (max1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (max2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9248, out_features=2, bias=True)
  (relu3): ReLU()
)

In [390]:
criterion=nn.MSELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [391]:
outputs

tensor([[-0.0946],
        [-0.1231],
        [-0.0848],
        [-0.0784],
        [-0.0872],
        [-0.0878],
        [-0.0995],
        [-0.0905]], grad_fn=<AddmmBackward>)

In [392]:
loss

tensor(0.6021, grad_fn=<MseLossBackward>)

In [393]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=2).eq(labels).sum().item()

In [397]:
ATCH_SIZE = 100
EPOCHS = 100
BATCH_SIZE=100

for epoch in range(EPOCHS):
    avg_loss = 0
    total_loss = 0
    total=0
    correct=0
    total_correct = 0
    for i in tqdm.tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 70, 70)
        batch_y = train_Y[i:i+BATCH_SIZE]

        model.zero_grad()

        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update
        
        avg_loss += loss.item() / len(train_X)
        
        model.eval()
#         valid_preds = np.zeros((len(train_X), num_classes))
        avg_val_loss = 0.
        
#         total_correct += get_num_correct(outputs, batch_y)
        
        _,prediction=torch.max(outputs.data,1)
        total+=batch_y.size(0)
        
        correct+=(prediction == batch_y).sum()
        accuray=100*correct / total

    print(f"Epoch: {epoch}. Loss: {loss.data}, Accuracy: {accuray}")

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (8) must match the size of tensor b (2) at non-singleton dimension 1

In [396]:
batch_X.shape,batch_y.shape

(torch.Size([4, 2, 70, 70]), torch.Size([8, 2]))

In [395]:
batch_y

tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.]])

In [359]:
train_Y[i:i+BATCH_SIZE]

tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.]])

In [349]:
prediction

tensor([0, 0, 0, 0, 0, 0, 0, 0])

In [366]:
batch_y.shape,batch_X.shape

(torch.Size([8, 2]), torch.Size([8, 1, 70, 70]))

In [304]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [352]:

trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

AttributeError: module 'torchvision.transforms' has no attribute 'RandomVerticalFlip'

In [355]:
transforms_dict = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.RandomHorizontalFlip(0.5),
        transforms.ToTensor(),
    ]),
}

TypeError: RandomHorizontalFlip() takes no arguments

In [357]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

AttributeError: module 'torchvision.transforms' has no attribute 'RandomVerticalFlip'

In [358]:
 model.predict(test_x)

AttributeError: 'CNN' object has no attribute 'predict'

In [398]:
batch_size=100
train_loader=torch.utils.data.DataLoader(dataset=train_norm,
                                        batch_size=batch_size,
                                        shuffle=True)

In [400]:
test_loader=torch.utils.data.DataLoader(dataset=test_covid,
                                       batch_size=batch_size,
                                       shuffle=False)

In [401]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1=nn.ReLU()
        self.max1=nn.MaxPool2d(kernel_size=2)
            
        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu2=nn.ReLU()
        self.max2=nn.MaxPool2d(kernel_size=2)
        
        self.fc1=nn.Linear(32*17*17,2)
        self.relu3=nn.ReLU()
        
        
    def forward(self,x):
        out=self.cnn1(x)
        out=self.relu1(out)
        out=self.max1(out)
        
        out=self.cnn2(out)
        out=self.relu2(out)
        out=self.max2(out)
        out=self.relu3(out)
        
        out=out.view(out.size(0),-1)
        out=self.fc1(out)
        return out
        
        
        
model=CNN()
model

CNN(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (max1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (max2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9248, out_features=2, bias=True)
  (relu3): ReLU()
)

In [402]:
criterion=nn.MSELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [408]:

n_iter=3000

In [411]:
train_loader.size

AttributeError: 'DataLoader' object has no attribute 'size'

In [413]:
num_epoch=n_iter/(100/batch_size)
num_epoch=int(num_epoch)
num_epoch

3000

In [414]:
iter=0
for i in range(num_epoch):
    for i,(images,labels) in enumerate(train_loader):
        images=Variable(images.view(-1,28*28))
        label=Variable(labels)
        
        optimizer.zero_grad()
        
        output=model(images)
        
        loss=criterion(output,labels)
        
        loss.backward()
        
        optimizer.step()
        
        iter+=1
        
        if iter%500==0:
            correct=0
            
            total=0
            
            for images,labels in test_loader:
                images=Variable(images.view(-1,28*28))
                
                output=model(images)
                
                _,prediction=torch.max(output.data,1)
                
                total+=labels.size(0)
                
                correct+=(prediction ==labels).sum()
                
            accuracy=100* correct / total
            
            print(f"iteration {iter} : loss {loss} : Accuracy {accuracy}")

ValueError: too many values to unpack (expected 2)